## TensorBoard

The following is how you do it on Google Colab. 

https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/

In other scenarios, after Training the model, run
```tensorboard --logdir=path/to/log-directory```

Tensorboard provides a good visualization tool for all the variables you like and works on a browser.


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-10-19 08:10:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.4.75.11, 52.44.92.122, 52.45.248.161, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.4.75.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]   5.11M  21.7MB/s    in 0.2s    

2018-10-19 08:10:49 (21.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Let's fire up tensorboard!

In [2]:
get_ipython().system_raw('tensorboard --logdir ./logs/09_tensorboard --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &') # run nrok to tunnel tensorboard to outside world
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://f13a3a3c.ngrok.io


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras import initializers
from keras.utils import np_utils
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [0]:
# Hyper parameters
batch_size = 128
nb_epoch = 100

# Parameters for MNIST dataset
nb_classes = 10

# Parameters for MLP
prob_drop_input = 0.2               # drop probability for dropout @ input layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

In [0]:
weight_initializer = initializers.RandomNormal(stddev=0.01)

In [6]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_Train = np_utils.to_categorical(y_train, nb_classes)
Y_Test = np_utils.to_categorical(y_test, nb_classes)

11493376/11490434 [==============================] - 0s 0us/step


In [7]:
# Multilayer Perceptron model
model = Sequential()
model.add(Dense(units=625, input_dim=784, kernel_initializer=weight_initializer, activation='sigmoid', name='dense1'))
model.add(Dropout(prob_drop_input, name='dropout1'))
model.add(Dense(units=625, input_dim=625, kernel_initializer=weight_initializer, activation='sigmoid', name='dense2'))
model.add(Dropout(prob_drop_hidden, name='dropout2'))
model.add(Dense(units=10, input_dim=625, kernel_initializer=weight_initializer, activation='softmax', name='dense3'))
model.compile(optimizer=RMSprop(lr=0.001, rho=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 625)               490625    
_________________________________________________________________
dropout1 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense2 (Dense)               (None, 625)               391250    
_________________________________________________________________
dropout2 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense3 (Dense)               (None, 10)                6260      
Total params: 888,135
Trainable params: 888,135
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Train
history = model.fit(X_train, Y_Train, validation_data=(X_test, Y_Test), epochs=nb_epoch, batch_size=batch_size, verbose=1,
                    callbacks=[TensorBoard(log_dir='./logs/09_tensorboard', histogram_freq=1)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 70us/step - loss: 0.6308 - acc: 0.7970 - val_loss: 0.2822 - val_acc: 0.9164
Epoch 2/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.2528 - acc: 0.9243 - val_loss: 0.1868 - val_acc: 0.9422
Epoch 3/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1869 - acc: 0.9444 - val_loss: 0.1408 - val_acc: 0.9555
Epoch 4/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1470 - acc: 0.9554 - val_loss: 0.1203 - val_acc: 0.9637
Epoch 5/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1235 - acc: 0.9627 - val_loss: 0.1176 - val_acc: 0.9629
Epoch 6/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1071 - acc: 0.9677 - val_loss: 0.0939 - val_acc: 0.9715
Epoch 7/100
60000/60000 [==============================] - 4s 64us/step - loss: 0.0946 - acc: 0.9708 - val_loss: 0.0865 - 

In [0]:
# Evaluate
evaluation = model.evaluate(X_test, Y_Test, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))